In [63]:
from functools import cmp_to_key
from copy import deepcopy 
import numpy as np
import pandas as pd
from bitalg.tests.test2 import Test
from bitalg.visualizer.main import Visualizer
import time

In [64]:
def det(a, b, c):
    return ((b[0] - a[0]) * (c[1] - b[1]) - (b[1]-a[1]) * (c[0] - b[0]) )

def points_orientation(a, b, c, eps = 0):
    computed_det = det(a, b, c)
    if computed_det > eps: return 1 #counter-clockwise turn
    elif computed_det < -eps: return -1
    return 0

def points_distance_square(a, b):
    return ((a[0]-b[0])**2 + (a[1]-b[1])**2)


def graham_algorithm(points):
    lowest_xy_point = min(points)

    def side_comp(x, y):
        orientation = points_orientation(lowest_xy_point, x, y)
        if orientation == 0: return points_distance_square(lowest_xy_point, x) - points_distance_square(lowest_xy_point, y)
        return -orientation

    points.sort(key=cmp_to_key(side_comp))

    new_points = [lowest_xy_point]

    for i in range(1, len(points)):
        if i == len(points)-1 or points_orientation(lowest_xy_point, points[i], points[i+1]) != 0: new_points.append(points[i])

    S = [new_points[0]]
    
    for i in range(1, len(new_points)):
        while len(S) > 1 and points_orientation(S[-2], S[-1], new_points[i]) != 1: S.pop()

        S.append(new_points[i])

    return S

In [128]:
CW = -1
CCW = 1
ON = 0 

class Node:
    def __init__(self, point):
        self.point = point
        self.p = None
        self.n = None

class Linked_List:
    def __init__(self):
        self.head = None
        self.tail = None

    def push_back(self, point, after = None):
        new_node = Node(point)
        if self.head == None:
            self.head = self.tail = new_node
        elif after == None or after == self.tail:
            new_node.p = self.tail
            self.tail.n = self.tail = new_node
        else:
            new_node.n = after.n
            new_node.p = after
            after.n.p = after.n = new_node
        return new_node

    def delete(self, to_be_deleted):
        if self.head == self.tail and self.head == to_be_deleted: self.head = self.tail = None

        if to_be_deleted == self.head: self.head = self.head.n; self.head.p = None
        elif to_be_deleted == self.tail: self.tail = self.tail.p; self.tail.n = None
        else: 
            to_be_deleted.p.n = to_be_deleted.n
            to_be_deleted.n.p = to_be_deleted.p
            to_be_deleted.n = to_be_deleted.p = None

        if self.head == None or self.tail == None: self.head = self.tail = None

    def push_front(self, point, before = None):
        new_node = Node(point)
        if not self.head:
            self.head = self.tail = new_node
        elif before == None or before == self.head:
            new_node.n = self.head
            self.head.p = self.head = new_node
        else:
            new_node.n = before
            new_node.p = before.p
            before.p.n = before.p = new_node
        return new_node
    
    def getNext(self, after): return self.head if after == self.tail else after.n

    def getPrevious(self, before): return self.tail if before == self.head else before.p


def Algorytm_przyrostowy(x):

    if len(x) <= 1: return x 

    x.sort()

    filtered_points = []
    for i in range(len(x)):
        if i != 0 and i != len(x)-1 and x[i][0] == x[i-1][0] and x[i][0] == x[i+1][0]: continue
        filtered_points.append(x[i])
    
    Otoczka = Linked_List()
    Otoczka.push_back(filtered_points[0])
    mid = Otoczka.push_back(filtered_points[1])

    for i in range(2, len(filtered_points)): 
        g_point = d_point = mid
        delete_queue = []
        
        while points_orientation(filtered_points[i], g_point.point, Otoczka.getNext(g_point).point) == CW or (points_orientation(filtered_points[i], g_point.point, Otoczka.getNext(g_point).point) == 0 and points_distance_square(filtered_points[i], g_point.point) < points_distance_square(filtered_points[i], Otoczka.getNext(g_point).point)):
            if g_point != mid: delete_queue.append(g_point)
            g_point = Otoczka.getNext(g_point)


        while points_orientation(filtered_points[i], d_point.point, Otoczka.getPrevious(d_point).point) == CCW or (points_orientation(filtered_points[i], d_point.point, Otoczka.getPrevious(d_point).point) == 0 and points_distance_square(filtered_points[i], d_point.point) < points_distance_square(filtered_points[i], Otoczka.getPrevious(d_point).point)):
            if d_point != mid: delete_queue.append(d_point)
            d_point = Otoczka.getPrevious(d_point)


        if g_point != mid and d_point != mid:
            delete_queue.append(mid)
        
        for node in delete_queue: 
            Otoczka.delete(node)

        mid = Otoczka.push_front(filtered_points[i], g_point)


    wynik = []
    head = Otoczka.head
    while head != None: 
        wynik.append(head.point)
        head = head.n


    return wynik

In [130]:
Test().runtest(2, Algorytm_przyrostowy)


Lab 2, task 2:
	Test 1: Passed
	Test 2: Passed
	Test 3: Passed
	Test 4: Passed
	Test 5: Passed
	Test 6: Passed
	Test 7: Passed
	Test 8: Passed
	Test 9: Passed
	Test 10: Passed
	Test 11: Passed
Result: 11/11
Time: 0.946s


In [5]:
Test().runtest(2, graham_algorithm)

Lab 2, task 2:
	Test 1: Passed
	Test 2: Passed
	Test 3: Passed
	Test 4: Passed
	Test 5: Passed
	Test 6: Passed
	Test 7: Passed
	Test 8: Passed
	Test 9: Passed
	Test 10: Passed
	Test 11: Passed
Result: 11/11
Time: 1.407s


In [55]:
points = [(-6.658355333673498, 7.46098547448939), (-1.4477138124303688, 9.894651318631613), (6.149610630085583, -7.885574747495481), (-8.116857338085737, -5.8409440121579435), (-6.174299723750963, 7.86625850841992), (9.43477869552029, 3.3143552867121038), (2.8753866681127316, -9.577690301363871), (3.1314609513515097, -9.497049663456579), (8.585063057908734, -5.1280300595580295), (7.409472589050965, 6.7156322079237185)]
ans, v = chan_algorithm(points)
v.add_point(ans, color='red')
v.show()

TypeError: 'float' object is not subscriptable